# ASA Real Estate Data EDA & Insights

This notebook explores the synthetic dataset generated for ASA Real Estate. It demonstrates the relational structure between Properties, Units, and Tenants, and provides visualizations to understand the market dynamics we are simulating.

## 1. Variable Definitions (Data Dictionary)

### Properties Table (`calibrated_properties.csv`)
| Variable | Type | Definition |
|----------|------|------------|
| `property_id` | String | Unique Key (FK in Units). Format: `PROP_XXX` |
| `name` | String | Name of the building/complex |
| `neighborhood` | String | Geo-location cluster (e.g., Tribeca, Harlem) |
| `class` | Char | Asset Class (A=Luxury/New, B=Standard, C=Older/Value) |

### Units Table (`calibrated_units.csv`)
| Variable | Type | Definition |
|----------|------|------------|
| `unit_id` | String | Unique Key (FK in Tenants). Format: `PROP_XXX_UXXX` |
| `property_id` | String | Foreign Key linking to Properties table |
| `type` | String | Bedroom count (Studio, 1BD, 2BD, 3BD) |
| `amenities` | String | List of features (Gym, Doorman, View) |
| `sqft` | Int | Square footage of the unit |
| `market_rent` | Int | Monthly rent price ($) |

### Tenants Table (`calibrated_tenants.csv`)
| Variable | Type | Definition |
|----------|------|------------|
| `tenant_id` | String | Unique ID for the resident |
| `unit_id` | String | Foreign Key linking to which unit they lease |
| `name` | String | Full name of the tenant |
| `income` | Int | Annual household income ($). Core metric for Affordability ratios. |
| `credit_score` | Int | FICO score (300-850). Core metric for Risk profiling. |
| `lease_start` | Date | When the current lease began. Used for Renewal logic. |

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_theme(style="whitegrid")

## 2. Load Data

In [ ]:
base_path = r"../src/data/synthetic/"

props = pd.read_csv(base_path + "calibrated_properties.csv")
units = pd.read_csv(base_path + "calibrated_units.csv")
tenants = pd.read_csv(base_path + "calibrated_tenants.csv")

print(f"Loaded {len(props)} Properties")
print(f"Loaded {len(units)} Units")
print(f"Loaded {len(tenants)} Tenants")

## 3. Relational Joins
We denormalize the tables into a single `Master_DF` to analyze connections (e.g., Does Building Class affect Tenant Income?)

In [ ]:
# Join Units -> Properties
units_props = units.merge(props, on='property_id', how='left')

# Join Tenants -> Units_Props
master_df = tenants.merge(units_props, on='unit_id', how='left')

master_df.head()

## 4. EDA & Insights

In [ ]:
# Insight 1: Rent Distribution by Neighborhood
plt.figure(figsize=(12, 6))
sns.boxplot(data=units_props, x='neighborhood', y='market_rent', hue='type')
plt.title("Rent Distribution by Neighborhood and Unit Type")
plt.show()

In [ ]:
# Insight 2: Income vs Rent (Affordability)
# Calculating Rent-to-Income Ratio
master_df['monthly_income'] = master_df['income'] / 12
master_df['rent_burden'] = master_df['market_rent'] / master_df['monthly_income']

plt.figure(figsize=(10, 6))
sns.scatterplot(data=master_df, x='market_rent', y='income', hue='class', alpha=0.6)
plt.title("Tenant Income vs. Market Rent (colored by Building Class)")
plt.show()

In [ ]:
# Insight 3: Credit Score by Building Class
plt.figure(figsize=(8, 5))
sns.violinplot(data=master_df, x='class', y='credit_score', order=['A', 'B', 'C'])
plt.title("Tenant Risk Profile (Credit Score) by Asset Class")
plt.show()

## 5. Summary Statistics
Key metrics to validate our synthetic engine.

In [ ]:
summary = master_df.groupby(['neighborhood', 'type']).agg({
    'market_rent': 'mean',
    'income': 'mean',
    'credit_score': 'mean',
    'rent_burden': 'mean'
}).round(2)

summary